# Aplicação do MobileNetV2 para classificação de imagens

Keras inclui uma série de redes pré-treinadas que você pode baixar e usar imediatamente [Tipos de modelo](https://keras.io/api/applications/). Um deles é o MobileNetV2, que foi treinado para classificar imagens.

Você pode simplesmente importar o MobileNetV2 de `keras.applications` e criar uma instância dele. Na primeira vez que você fizer isso, a rede será baixada da Internet.

Se você não passar nenhuma palavra-chave para MobileNetV2(), a rede terá pesos aleatórios; ou seja, você obterá a arquitetura da rede, mas não os pesos e, portanto, terá que treiná-la você mesmo. Ao especificar `weights='imagenet'`, você indica que deseja que a rede seja pré-treinada.

In [35]:
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import numpy as np
from imageio import imread
from google.colab import files as FILE
import os
import requests
import tensorflow as tf

# Download de uma imagem na internet
# DownURL = 'https://i0.wp.com/ethosanimal.com.br/wp-content/uploads/2015/07/cao-em-harmonia-cao-feliz-ethos-animal-comportamento-animal-aplicado-adestramento-helena-truksa-.jpg?w=728&ssl=1'
DownURL = 'https://www.plazahoteis.com.br/wp-content/uploads/2018/10/cerveja-artesanal-blumenau-out18.jpg'

img_data = requests.get(DownURL).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)

# Modelo MobileNetV2
model = MobileNetV2(weights='imagenet')

As redes neurais são exigentes quando se trata do tipo de entrada que esperam. Se você fornecer entrada em um formato que a rede não espera, as previsões não farão sentido (se a estrutura da entrada for superficialmente compatível com a rede) ou o código simplesmente falhará (se a estrutura da entrada é incompatível).

O MobileNetV2 espera imagens de 224 × 224 pixels com três canais de cores. Em outras palavras, espera entrada de forma (224, 224, 3). Você pode passar várias imagens de uma vez para o modelo, assim como passamos vários números para nosso modelo no tutorial anterior. Isso significa que a entrada deve, na verdade, consistir em quatro dimensões, onde a primeira dimensão corresponde ao índice da imagem.

Aqui passamos apenas uma única imagem para o modelo, o que significa que a primeira dimensão da entrada é de tamanho 1.

In [27]:
# Ler a imagem salva na raiz
image_input = imread('/content/image_name.jpg')

# Conveter a imagem para formato do MobileNetV2
image = tf.image.resize(image_input, (224, 224))

data = np.empty((1, 224, 224, 3))
data[0] = image

Quando você carrega uma imagem com `imread()`, os valores de pixel estão no intervalo de 0 a 255. No entanto, o MobileNetV2 espera que os valores de pixel estejam na faixa de -1 a 1. Você mesmo pode realizar essa transformação, mas o Keras também fornece funções de pré-processamento:

In [42]:
data = preprocess_input(data)

# Classificação da Imagem

As previsões são retornadas como um array bidimensional onde o primeiro eixo corresponde às observações (imagens, neste caso). Como alimentamos apenas uma única imagem no modelo, a primeira dimensão é de tamanho 1. O segundo eixo corresponde aos neurônios de saída do modelo. Como o MobileNetV2 tem 1.000 neurônios na camada de saída, a segunda dimensão é de tamanho 1.000.

Podemos verificar qual neurônio de saída estava mais ativo usando np.argmax(), que fornece o índice do valor mais alto:

In [38]:
predictions = model.predict(data)
print('Shape: {}'.format(predictions.shape))


output_neuron = np.argmax(predictions[0])
print('Neuronio mais ativo: {} ({:.2f}%)'.format(
    output_neuron,
    100 * predictions[0][output_neuron]
))


Shape: (1, 1000)
Neuronio mais ativo: 818 (21.71%)


No entanto, saber que o neurônio 818 se tornou o mais ativo em resposta à imagem lida não é, por si só, muito informativo, pois também precisamos saber a qual categoria ele corresponde. Felizmente, o Keras fornece uma função que mapeia automaticamente as previsões em pontuações para as categorias correspondentes:

In [39]:
for name, desc, score in decode_predictions(predictions)[0]:
    print('- {} ({:.2f}%%)'.format(desc, 100 * score))


- spotlight (21.71%%)
- digital_clock (4.06%%)
- theater_curtain (3.18%%)
- matchstick (3.11%%)
- lampshade (2.54%%)
